<h1 style="text-align:center">Model Development</h1>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
path = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DA0101EN/automobileEDA.csv"
df = pd.read_csv(path)
df.head(5)

,symboling,normalized-losses,make,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,...,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,city-L/100km,horsepower-binned,diesel,gas
0,3,122,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,...,9.0,111.0,5000.0,21,27,13495.0,11.190476,Medium,0,1
1,3,122,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,...,9.0,111.0,5000.0,21,27,16500.0,11.190476,Medium,0,1
2,1,122,alfa-romero,std,two,hatchback,rwd,front,94.5,0.822681,...,9.0,154.0,5000.0,19,26,16500.0,12.368421,Medium,0,1
3,2,164,audi,std,four,sedan,fwd,front,99.8,0.848630,...,10.0,102.0,5500.0,24,30,13950.0,9.791667,Medium,0,1
4,2,164,audi,std,four,sedan,4wd,front,99.4,0.848630,...,8.0,115.0,5500.0,18,22,17450.0,13.055556,Medium,0,1


### 1. Linear Regression Model :

In [3]:
from sklearn.linear_model import LinearRegression

In [4]:
lr = LinearRegression()

In [5]:
x = df[["highway-mpg"]]
y = df[["price"]]

In [6]:
lr.fit(x,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [9]:
yhat = lr.predict(x)
yhat[0:5]

array([[16236.50464347],
       [16236.50464347],
       [17058.23802179],
       [13771.3045085 ],
       [20345.17153508]])

In [19]:
intercept = lr.intercept_
coef = lr.coef_
print("Intercept : ",intercept,"Coefficient : ",coef)

Intercept :  [38423.30585816] Coefficient :  [[-821.73337832]]


In [18]:
price = intercept + coef * 27
price

array([[16236.50464347]])

Final estimate linear model :
#### price = 38423.30585816 + (-821.73337832) * highway-mpg

### 2. Multiple Linear Regression

In [23]:
z = df[['horsepower','curb-weight','engine-size','highway-mpg']]

In [25]:
lr.fit(z,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [26]:
intercept = lr.intercept_
coef = lr.coef_
print("Intercept : ",intercept," Coefficient : ",coef)

Intercept :  [-15806.62462633]  Coefficient :  [[53.49574423  4.70770099 81.53026382 36.05748882]]


Final estimate MLR model :
#### price  = -15806.62462633 + 53.49574423 * horsepower + 4.70770099 * curb-weight + 81.53026382 * engine-size + 36.05748882 * highway-mpg